1. If not in an optimization procedure and in REVDSD optimization, show the imaginary mode on the right

2. If in optimization procedure, add option to submit a scan. Should have 2 input options for atom indices as well as print the current bond length and give a space to indicate the start length, end length, number of points, charge, and multiplicity

3. When looking at scan energy

In [ ]:
import ipywidgets as widgets
import sys
from pathlib import Path

sys.path.append("../src/protopilot")
from protopilot.io import optimize

import protopilot.visualizations.xyz as visualize_xyz
import protopilot.visualizations.smiles as visualize_smiles

# --- Header widgets ---
smiles_input = widgets.Text(placeholder="Enter SMILES...", description="SMILES:")
procedure_dropdown = widgets.Dropdown(options=["NA"], description="Procedure:")
method_dropdown = widgets.Dropdown(options=["NA"], description="Method:")
xyz_dropdown = widgets.Dropdown(options=["NA"], description="XYZ:")
dropdowns_box = widgets.HBox([procedure_dropdown, method_dropdown, xyz_dropdown])
dropdowns_box.layout.visibility = "hidden"

# --- Lower header (button row) ---
submit_path_btn = widgets.Button(description="Submit path")
lower_header_box = widgets.HBox([submit_path_btn])
lower_header_box.layout.visibility = "hidden"

# --- Header layout with box and color ---
header_inner = widgets.VBox(
    [widgets.HBox([smiles_input]), dropdowns_box, lower_header_box]
)
header_box = widgets.Box(
    [header_inner],
    layout=widgets.Layout(
        border="solid 2px #2196F3",
        padding="10px",
        margin="5px 0px 10px 0px",
        background_color="#e3f2fd",
        width="100%",
    ),
)
header_box.layout.background_color = "#e3f2fd"  # light blue

# --- Output widgets ---
xyz_output = widgets.Output()
zpv_label = widgets.Label(value="Zero Point Vibrational Energy: --")
spc_label = widgets.Label(value="Single Point Calculation Energy: --")
energies_box = widgets.VBox([zpv_label, spc_label])
energies_box.layout.visibility = "hidden"
submit_optimization_btn = widgets.Button(description="Submit optimization")

# --- Left column header ---
structure_header = widgets.HTML(
    '<h2 style="text-align:center; font-size:1.7em; margin-bottom:10px;">STRUCTURE</h2>'
)

# --- Center column widgets ---
path_plot_output = widgets.Output()
path_plot_output.layout.visibility = "hidden"
atom_indices_box = widgets.HBox(
    [
        widgets.Label("Atom 1 Index:"),
        widgets.BoundedIntText(min=0, max=999, value=0),
        widgets.Label("Atom 2 Index:"),
        widgets.BoundedIntText(min=0, max=999, value=1),
        widgets.Button(description="Submit Indices"),
    ]
)
atom_indices_box.layout.visibility = "hidden"
vib_mode_output = widgets.Output()
vib_mode_output.layout.visibility = "hidden"

# --- Layout containers with colored boxes ---
# Left column: light green
# Center column: light yellow
left_col_inner = widgets.VBox(
    [structure_header, xyz_output]
)  # energies_box or submit_optimization_btn will be added dynamically
left_col = widgets.Box(
    [left_col_inner],
    layout=widgets.Layout(
        border="solid 2px #43a047",
        padding="10px",
        margin="0px 10px 0px 0px",
        background_color="#e8f5e9",
        min_width="350px",
    ),
)
left_col.layout.background_color = "#e8f5e9"  # light green
right_col_inner = widgets.VBox([])  # will be set dynamically
right_col = widgets.Box(
    [right_col_inner],
    layout=widgets.Layout(
        border="solid 2px #fbc02d",
        padding="10px",
        margin="0px",
        background_color="#fffde7",
        min_width="600px",
    ),
)
right_col.layout.background_color = "#fffde7"  # light yellow
body = widgets.HBox([left_col, right_col])

display(header_box, body)


# --- Update functions ---
def update_smiles(change):
    xyz_output.clear_output()
    dropdowns_box.layout.visibility = "hidden"
    lower_header_box.layout.visibility = "hidden"
    smiles_str = smiles_input.value
    with xyz_output:
        xyz_path = visualize_xyz.find_xyz(smiles_str=smiles_str)
        if xyz_path is None:
            try:
                visualize_smiles.visualize_smiles(smiles_str)
            except Exception:
                pass
            left_col_inner.children = [
                structure_header,
                xyz_output,
                submit_optimization_btn,
            ]
        else:
            procedure_dropdown.options = visualize_xyz.find_dirs(smiles_str=smiles_str)
            dropdowns_box.layout.visibility = "visible"
            left_col_inner.children = [
                structure_header,
                xyz_output,
            ]  # will be updated by update_xyz


def update_procedure(change):
    run_dir = Path(procedure_dropdown.value) / "run"
    method_options = [(d.name, d) for d in run_dir.iterdir() if d.is_dir()]
    method_dropdown.options = method_options
    method_dropdown.layout.visibility = "visible"
    # Show lower header buttons depending on procedure
    if "Path" in procedure_dropdown.value.name:
        lower_header_box.children = [submit_path_btn]
        lower_header_box.layout.visibility = "visible"
    else:
        lower_header_box.layout.visibility = "hidden"


def update_method(change):
    method_dir = Path(method_dropdown.value)
    xyz_options = [(x.name, x) for x in method_dir.rglob("*.xyz")]
    xyz_dropdown.options = xyz_options
    xyz_dropdown.layout.visibility = "visible"
    # Center column logic
    if "Scan" in method_dir.name:
        path_plot_output.layout.visibility = "visible"
        atom_indices_box.layout.visibility = "hidden"
        vib_mode_output.layout.visibility = "hidden"
        right_col_inner.children = [path_plot_output]
        with path_plot_output:
            path_plot_output.clear_output()
            visualize_xyz.allxyz_energies(method_dir)
    elif "Optimization" in procedure_dropdown.value.name:
        path_plot_output.layout.visibility = "hidden"
        atom_indices_box.layout.visibility = "visible"
        vib_mode_output.layout.visibility = "hidden"
        right_col_inner.children = [atom_indices_box]
    else:
        path_plot_output.layout.visibility = "hidden"
        atom_indices_box.layout.visibility = "hidden"
        vib_mode_output.layout.visibility = "visible"
        right_col_inner.children = [vib_mode_output]


def update_xyz(change):
    xyz_output.clear_output()
    run_dir = Path(procedure_dropdown.value) / "run"
    xyz_path = Path(xyz_dropdown.value)
    with xyz_output:
        if any(s in xyz_path.name for s in ["trj", ".allxyz"]):
            visualize_xyz.animate_xyz(xyz_path)
        else:
            visualize_xyz.labelled_3d_structure(xyz_path)
    # --- Update energies ---
    revdsd_logs = [x for x in (run_dir / "REVDSD").rglob("REVDSD.log")]
    zpv_energies = []
    for log in revdsd_logs:
        with open(str(log), "r") as f:
            zpv_energies += (
                line.split("Eh")[1].strip() for line in f if "Zero point energy" in line
            )
    ccsdt_logs = [x for x in (run_dir / "CCSDT").rglob("CCSDT.log")]
    spc_energies = []
    for log in ccsdt_logs:
        with open(str(log), "r") as f:
            spc_energies += (
                line.split("ENERGY")[1].strip()
                for line in f
                if "FINAL SINGLE POINT ENERGY" in line
            )
    if zpv_energies or spc_energies:
        zpv_label.value = (
            "ZPV: " + ",".join(e for e in zpv_energies)
            if zpv_energies
            else "Zero Point Vibrational Energy: --"
        )
        spc_label.value = (
            "SPC (hartrees): " + ",".join(e for e in spc_energies)
            if spc_energies
            else "Single Point Calculation Energy: --"
        )
        energies_box.layout.visibility = "visible"
        left_col_inner.children = [structure_header, xyz_output, energies_box]
    else:
        energies_box.layout.visibility = "hidden"
        left_col_inner.children = [
            structure_header,
            xyz_output,
            submit_optimization_btn,
        ]


def submit_opt(click):
    optimize.write(smiles_string=smiles_input.value)
    # Replace button with a copyable Text widget
    bash_cmd_widget = widgets.Label(
        value="Submitted!", disabled=False, layout=widgets.Layout(width="95%")
    )
    left_col_inner.children = [structure_header, xyz_output, bash_cmd_widget]


smiles_input.observe(update_smiles, names="value")
procedure_dropdown.observe(update_procedure, names="value")
method_dropdown.observe(update_method, names="value")
xyz_dropdown.observe(update_xyz, names="value")
submit_optimization_btn.on_click(submit_opt)

Box(children=(VBox(children=(HBox(children=(Text(value='', description='SMILES:', placeholder='Enter SMILES...…

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

import sys

sys.path.append("../src/protopilot")


mol = Chem.MolFromSmiles(smiles)
img = Draw.MolToImage(
    mol,
    molsPerRow=1,
    subImgSize=(200, 200),
)

display(img)

NameError: name 'smiles' is not defined

## Define the transition state smiles string and the path to the optimized structure preceding the transition

In [34]:
from pathlib import Path
import sys

sys.path.append(str(Path.home() / "C5O-Kinetics"))
import altair as alt
import numpy as np
import pandas as pd
import py3Dmol
import pyparsing as pp
from pyparsing import pyparsing_common as ppc
from rdkit import Chem
from src.protopilot.io.transition import write_opt, write_scan
from src.protopilot.visualizations.smiles import visualize_smiles
from src.protopilot.visualizations.xyz import animate_xyz, labelled_3d_structure

import glob as glob
import re
import sys

In [31]:
trans_smiles = "C1=C~[O]1"
folder_name = "_CH2_CdblO"

### Visualize the transition state and optimized structure, then define the scan coordinates

In [35]:
trans_smiles = trans_smiles.replace("/", "~")
allxyz_paths = [
    path
    for path in Path(
        f"/home/tns97255/C5O-Kinetics/calc/{folder_name}/Optimization/run/REVDSD/"
    ).rglob("REVDSD.xyz")
]
if len(allxyz_paths) > 1:
    raise ValueError(
        f"Multiple XTB.allxyz files found in {Path(f'/home/tns97255/C5O-Kinetics/calc/{folder_name}/Optimization/')}. Please check the directory."
    )
else:
    inter_xyz = Path(allxyz_paths[0])

visualize_smiles(trans_smiles)
labelled_3d_structure(inter_xyz)


NameError: name 'trans_smiles' is not defined

In [7]:
scan_coordinates = "B" + " " + "0 2"  # geom scan coordinates
charge = 0
multiplicity = 2  # 1 if neutral, 2+ if radical
min_scan_dist = 0.1  # minimum distance for scan
max_scan_dist = 1.5  # maximum distance for scan

### Read the equilibrium difference between two indices (optional)

In [9]:
equil_xyz = Path(
    "/home/tns97255/C5O-Kinetics/calc/_CH_1CO1/Optimization/run/REVDSD/40242785/REVDSD.xyz"
)

In [10]:
_, a1, a2 = scan_coordinates.split(" ")
idx1 = int(a1)
idx2 = int(a2)

# --- Get 3D coordinates
with open(equil_xyz) as f:
    lines = f.readlines()
coords = []
for line in lines[2:]:  # skip first two lines of xyz
    parts = line.split()
    if len(parts) >= 4:
        coords.append([float(parts[1]), float(parts[2]), float(parts[3])])
coords = np.array(coords)
bond_len = np.linalg.norm(coords[idx1, :] - coords[idx2, :])

print(f"{bond_len:.2f}")

1.37


### Generate the scan .inp file

In [28]:
mol = Chem.MolFromXYZFile(str(inter_xyz))
_, a1, a2 = scan_coordinates.split(" ")
idx1 = int(a1)
idx2 = int(a2)

n_steps = abs(max_scan_dist - min_scan_dist) // 0.05 + 1

# --- Get 3D coordinates
with open(inter_xyz) as f:
    lines = f.readlines()
coords = []
for line in lines[2:]:  # skip first two lines of xyz
    parts = line.split()
    if len(parts) >= 4:
        coords.append([float(parts[1]), float(parts[2]), float(parts[3])])
coords = np.array(coords)
bond_len = np.linalg.norm(coords[idx1, :] - coords[idx2, :])

rxn_coordinates = (
    scan_coordinates
    + f" = {(bond_len + min_scan_dist):.2f}, {(bond_len + max_scan_dist):.2f}, {n_steps}\n"
)
print(rxn_coordinates)
write_scan(
    intermediate_xyz=str(inter_xyz),
    trans_smiles=trans_smiles,
    scan_coordinates=scan_coordinates,
    rxn_coordinates=rxn_coordinates,
    charge=charge,
    multiplicity=multiplicity,
)

B 0 2 = 2.44, 1.24, 25.0

pixi run run_scan /home/tns97255/C5O-Kinetics/calc/_CH2_CdblO/B_0_2/run/REVDSD_Scan


## Visualize to determine which transition structure is highest energy

In [37]:
from pathlib import Path

max_step = None
xyz_path = Path("/home/tns97255/C5O-Kinetics/calc/_CH_1CO1/Test1")

In [38]:
import pyparsing as pp

try:
    work_dir = xyz_path / "run/REVDSD_Scan/"
except TypeError:
    par_dir = Path(str(inter_xyz).split("Optimization")[0])
    work_dir = par_dir / f"{scan_coordinates.replace(' ', '_')}/run/REVDSD_Scan/"
allxyz_paths = [path for path in work_dir.rglob("REVDSD.allxyz")]
if len(allxyz_paths) > 1:
    raise ValueError(
        f"Multiple XTB.allxyz files found in {work_dir}. Please check the directory."
    )
else:
    allxyz_path = allxyz_paths[0]

comment = pp.Group(
    pp.Keyword("Scan Step")
    + ppc.integer("index")
    + pp.Keyword("E")
    + ppc.fnumber("energy")
)
expr = pp.OneOrMore(pp.SkipTo(comment, include=True))
text = re.sub(">\n", "", allxyz_path.read_text())

indices = []
energies = []
results = expr.parse_string(text)
for result in results[1::2]:
    index, energy = result[1:4:2]
    indices.append(index)
    energies.append(energy)

df = pd.DataFrame(
    {
        "index": indices,
        "energy": energies,
    }
)

print("Plot of energy vs. index")
alt.Chart(df).mark_point().encode(
    x="index",
    y=alt.Y("energy", scale=alt.Scale(zero=False)),
).show()

if max_step is None:
    max_step = df["index"][df["energy"].idxmax()]

print(f"Maximum energy step: {max_step}")

max_scan_xyz_path = glob.glob(str(allxyz_path.parent / f"REVDSD.*{max_step:03d}.xyz"))[
    0
]

xyz = Chem.MolToXYZBlock(Chem.MolFromXYZFile(max_scan_xyz_path))
smiles = Chem.MolToSmiles(Chem.MolFromXYZBlock(xyz))

natoms = int(allxyz_path.read_text().split()[0])

viewer = py3Dmol.view()
viewer.addModel(xyz, "xyz")
viewer.setStyle({"stick": {}, "sphere": {"scale": 0.3}})

for i in range(natoms):
    viewer.addLabel(
        i,
        {
            "backgroundOpacity": 0,
            "fontColor": "blue",
            "alignment": "center",
            "inFront": True,
        },
        {"index": i},
    )

viewer.zoomTo()
print(f"Visualization of the highest energy structure (index {max_step})")
viewer.show()

### Automatically find the path to the scan coordinates

Plot of energy vs. index


alt.Chart(...)

Maximum energy step: 40
Visualization of the highest energy structure (index 40)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Write the optimization files

In [43]:
write_opt(
    transition_xyz=max_scan_xyz_path,
    trans_smiles=trans_smiles,
    charge=charge,
    multiplicity=multiplicity,
)

bash /home/tns97255/C5O-Kinetics/calc/_CH2_CCC1CO1/B_1_2/run/submit_opt.sh


### Visualize the imaginary vibrational mode

In [41]:
xyz_path = "/home/tns97255/C5O-Kinetics/calc/_CH_1CO1/Test1/run/REVDSD_Scan/40296259/REVDSD_trj.xyz"

In [42]:
animate_xyz(xyz_path)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.